In [1]:
import sqlite3 as dbms

conn = dbms.connect("students.sqlite3")  # Как оригинально!
cursor = conn.cursor()

with open("11.drop_ddl.sql", 'r', encoding='utf-8') as f:
    drop_ddl = f.read()
with open("11.create_ddl.sql", 'r', encoding='utf-8') as f:
    create_ddl = f.read()
    
if True:
    cursor.executescript(drop_ddl)
    conn.commit()
if True:
    cursor.executescript(create_ddl)
    conn.commit()

In [2]:
import sqlalchemy
from sqlalchemy import Table, Column, Integer, String, MetaData, ForeignKey
from sqlalchemy.orm import sessionmaker, relationship
from sqlalchemy.ext.declarative import declarative_base

DeclBase = declarative_base()

engine = sqlalchemy.create_engine('sqlite:///students.sqlite3', echo=False)  # echo=True для логгинга
Session = sessionmaker(bind=engine)
session = Session()

null=True
class Program(DeclBase):
    __tablename__ = 'programs'
    id = Column(Integer, primary_key=True)
    name = Column(String)

    students = relationship("Student", backref="program")
    programs_courses = relationship("Program_course", backref="program")

    def __init__(self, name):
        self.name = name
        
class Student(DeclBase):
    __tablename__ = 'students'
    id = Column(Integer, primary_key=True)
    card = Column(String)
    surname = Column(String)
    name = Column(String)
    patronymic = Column(String)
    
    program_id = Column(Integer, ForeignKey('programs.id'))
    marks = relationship("Mark", backref="student")
    
    def __init__(self, card, surname, name, patronymic, program):
        self.card = card
        self.surname = surname
        self.name = name
        self.patronymic = patronymic
        self.program = program

class Course(DeclBase):
    __tablename__ = 'courses'
    id = Column(Integer, primary_key=True)
    name = Column(String)
    
    programs_courses = relationship("Program_course", backref="course")
    marks = relationship("Mark", backref="course")
    
    def __init__(self, name):
        self.name = name
        
class Program_course(DeclBase):
    __tablename__ = 'programs_courses'
    semester_number = Column(Integer, primary_key=True)
    
    course_id = Column(Integer, ForeignKey('courses.id'), primary_key=True)
    program_id = Column(Integer, ForeignKey('programs.id'), primary_key=True)
    
    def __init__(self, semester_number, course, program):
        self.semester_number = semester_number
        self.course = course
        self.program = program
        
class Mark(DeclBase):
    __tablename__ = 'marks'
    mark = Column(Integer)
    
    course_id = Column(Integer, ForeignKey('courses.id'), primary_key=True)
    student_id = Column(Integer, ForeignKey('students.id'), primary_key=True)
    
    def __init__(self, mark, student, course):
        self.mark = mark
        self.student = student
        self.course = course

se = Program("Программная инженерия")

st1 = Student("002020", "Иванов", "Пётр", "Сидорович", se)
st2 = Student("002019", "Петрова", "Исидора", "Ивановна", se)
st3 = Student("002018", "Зеленый", "Фикус", "Дмитриевич", se)
c1 = Course("Английский язык")
pc1 = Program_course("11", c1, se)
m1 = Mark("4", st1, c1)
m2 = Mark("5", st2, c1)
m3 = Mark("5", st3, c1)


session.add_all([st1, st2, st3, c1, pc1, m1, m2, m3]) 
session.commit()

import sys

print("Программы и студенты")
for p in session.query(Program):
    print("Программа: ", p.name)
    for s in p.students:
        print("- ", s.name, s.patronymic)
    for s in p.programs_courses:
        print("Номер семестра:", s.semester_number)
for p in session.query(Student):
    print("Студент: ", p.name)
    for s in p.marks:
        print("- ", s.mark)

Программы и студенты
Программа:  Программная инженерия
-  Пётр Сидорович
-  Исидора Ивановна
Номер семестра: 14
Программа:  Программная инженерия
-  Пётр Сидорович
-  Исидора Ивановна
-  Фикус -
Номер семестра: 11
Программа:  Программная инженерия
-  Пётр Сидорович
-  Исидора Ивановна
-  Фикус -
Номер семестра: 11
Программа:  Программная инженерия
-  Пётр Сидорович
-  Исидора Ивановна
-  Фикус Дмитриевич
Номер семестра: 11
Студент:  Пётр
-  5
Студент:  Исидора
-  8
Студент:  Пётр
-  4
Студент:  Исидора
-  5
Студент:  Фикус
-  5
Студент:  Пётр
-  4
Студент:  Исидора
-  5
Студент:  Фикус
-  5
Студент:  Пётр
-  4
Студент:  Исидора
-  5
Студент:  Фикус
-  5
